<a href="https://colab.research.google.com/github/SayanGhosh1489/BingNewsSearch/blob/main/Bing_News_Summarizer_with_Azure_OpennAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [157]:
from google.colab import userdata
from openai import AzureOpenAI
import os
import requests
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from pprint import pprint
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Constants and variables

In [156]:
key = userdata.get('AZURE-OPENAI-KEY')
endpoint = userdata.get('AZURE-OPENAI-ENDPOINT')
deployment = 'gpt-4'

bing_news_key = userdata.get('BING-NEWS-KEY')
bing_news_api = "https://api.bing.microsoft.com/v7.0/news/search"
search_term = "tata motors share"
count = 10
MAX_TOKENS = 8192  # Model's max tokens
RESERVED_TOKENS = 800  # For response, system messages, etc.
available_tokens = MAX_TOKENS - RESERVED_TOKENS

In [119]:
headers = {"Ocp-Apim-Subscription-Key" : bing_news_key}

In [120]:
#getting news from bing news

params  = { 'q' : search_term,
    "textDecorations": True,
    "textFormat": "HTML",
    "freshness" : "Month",
    "mkt" : 'en-IN',
    'count' : count,
    'sortBy': 'date'}


response = requests.get(bing_news_api,headers= headers, params= params)

In [121]:
search_result = response.json()
search_result.keys()

dict_keys(['_type', 'readLink', 'queryContext', 'totalEstimatedMatches', 'sort', 'value'])

In [124]:
urls = []
for i, term in enumerate(search_result['value']):
  print(i, term['datePublished'])
  print(term['url'])
  urls.append(term['url'])
  print("=="*10)

0 2025-01-27T14:22:00.0000000Z
https://www.dnaindia.com/business/report-stock-markets-fall-for-2nd-day-mukesh-ambani-reliance-loses-rs-22937-crore-share-market-news-bse-nse-3129257
1 2025-01-27T12:41:00.0000000Z
https://www.businesstoday.in/markets/stocks/story/tata-motors-shares-hit-one-year-low-should-investors-consider-averaging-462173-2025-01-27
2 2025-01-27T10:17:00.0000000Z
https://www.goodreturns.in/news/tata-group-stock-not-tata-motors-2-hidden-tata-stocks-are-worst-performers-in-30-days-down-14-23-buy-1401799.html
3 2025-01-27T08:46:00.0000000Z
https://economictimes.indiatimes.com/markets/stocks/stock-liveblog/tata-motors-share-price-live-27-jan-2025/liveblog/117589541.cms
4 2025-01-27T08:04:00.0000000Z
https://www.msn.com/en-in/autos/news/toyota-suv-mpv-sales-see-highest-growth-of-43-percent-in-april-december-2024/ar-AA1xV5ZR
5 2025-01-27T07:09:00.0000000Z
https://www.goodreturns.in/news/tata-motors-share-falls-2-89-hits-52-week-low-m-cap-falls-to-rs-2-64l-cr-a-buying-opportu

## Collecting all the text

In [128]:
final_text = " "
for url in urls:
  response1 = requests.get(url)
  result = BeautifulSoup(response1.content, 'html.parser')
  text = result.find('body').get_text()

  final_text = final_text+text

## Text Cleaning

In [154]:
def clean_text(text):
    whitespace = re.compile(r"\s+")
    user = re.compile(r"(?i)@[a-z0-9_]+")
    urls = re.compile(r"http[s]?://\S+")
    text = whitespace.sub(' ', text)
    text = user.sub('', text)
    text = re.sub(r"\[[^()]*\]","", text)
    text = re.sub(r'[^\w\s]','',text)
    text = re.sub(r"(?:@\S*|#\S*|http(?=.*://)\S*)", "", text)
    text = urls.sub('', text)
    text = text.lower()

    # removing stop-words
    STOPWORDS = set(stopwords.words('english'))
    lemmatizer = PorterStemmer()
    words = [lemmatizer.stem(word) for word in text.split() if word not in STOPWORDS]
    words = words[:8190]
    print(len(words))
    final =  ' '.join(words)
    print(len(final))
    return final


In [155]:
cleaned_text = clean_text(final_text)

7081
46095


In [148]:
# Truncate the cleaned text to fit the available tokens
cleaned_text = ' '.join(cleaned_text.split()[:available_tokens // 2])

In [149]:
client = AzureOpenAI(azure_endpoint = endpoint,
                     api_key =  key,
                     api_version = "2024-05-01-preview")

In [151]:
chat_prompt = [
    {
        "role": "system",
        "content": [
            {
                "type": "text",
                "text": f"You are an AI assistant to sumarize the provided text and limited to {search_term}."
            }
        ]
    }
]

# user_text = input("Enter your question: ")

user_prompt = {

               "role": "user",
               "content" : [
                   {
                       "type" : "text",
                       "text" : cleaned_text
                   }
               ]
}

chat_prompt.append(user_prompt)

# Generate the completion
completion = client.chat.completions.create(
    model=deployment,
    messages=chat_prompt,
    max_tokens=800,
    temperature=0.7,
    top_p=0.4,
    frequency_penalty=0.13,
    presence_penalty=0.3,
    stop=None,
    stream=False
)

The share price of Tata Motors has hit a one-year low, slipping 3.33% to a 52-week low value of Rs 709.40. The stock eventually settled at a decrease of 2.83% at Rs 713.15. Over the past six months, the price has corrected by 36.54%. The stock saw high trading volumes on the BSE with around 7.62 lakh shares changing hands. The turnover for the counter was Rs 547.3 crore, and it commands a market capitalisation of Rs 262519.42 crore. Market experts suggest that the technical outlook for Tata Motors looks bleak with immediate support seen in the Rs 700-680 range.


In [153]:
pprint(completion.choices[0].message.content)

('The share price of Tata Motors has hit a one-year low, slipping 3.33% to a '
 '52-week low value of Rs 709.40. The stock eventually settled at a decrease '
 'of 2.83% at Rs 713.15. Over the past six months, the price has corrected by '
 '36.54%. The stock saw high trading volumes on the BSE with around 7.62 lakh '
 'shares changing hands. The turnover for the counter was Rs 547.3 crore, and '
 'it commands a market capitalisation of Rs 262519.42 crore. Market experts '
 'suggest that the technical outlook for Tata Motors looks bleak with '
 'immediate support seen in the Rs 700-680 range.')
